In [1]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [91]:
import tensorflow as ts
from tensorflow import keras
import os
import numpy as np
from os import listdir
from skimage.io import imread
from skimage.transform import resize
from sklearn.model_selection import train_test_split
grayscale_images = True
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

In [92]:
dataset_path = "/content/drive/MyDrive/Dataset"
def get_img(dataset_path):
    img = imread(dataset_path)
    img = resize(img, (64, 64, 1 if grayscale_images else 3))
    return img
  

In [93]:
from keras import backend as evaluatee

def recall_m(y_true, y_pred):
    true_positives = evaluatee.sum(evaluatee.round(evaluatee.clip(y_true * y_pred, 0, 1)))
    possible_positives = evaluatee.sum(evaluatee.round(evaluatee.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + evaluatee.epsilon())
    return recall
def precision_m(y_true, y_pred):
    true_positives = evaluatee.sum(evaluatee.round(evaluatee.clip(y_true * y_pred, 0, 1)))
    predicted_positives = evaluatee.sum(evaluatee.round(evaluatee.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + evaluatee.epsilon())
    return precision

def f_score(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+evaluatee.epsilon()))

In [94]:
labels = listdir(dataset_path) # Geting labels
X = []
Y = []
for i, label in enumerate(labels):
    datas_path = dataset_path+'/'+label
    for data in listdir(datas_path):
        img = get_img(datas_path + '/' + data)
        X.append(img)
        Y.append(i)
X = np.array(X).astype('float32')/255.0
Y = np.array(Y).astype('float32')
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
print(x_train.shape)

(1649, 64, 64, 1)


In [95]:
x_train = x_train.reshape(len(x_train),64*64)
x_test = x_test.reshape(len(x_test),64*64)

In [109]:
def NN1():
    model = keras.Sequential([
    keras.layers.Dense(512, input_shape=(64*64,), activation='relu'),
    keras.layers.Dense(160, activation='relu'),
    keras.layers.Dense(10, activation='sigmoid')
])

    model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy',f_score,precision_m, recall_m])

    return model

In [110]:
model1 = NN1()
model1.fit(x_train, y_train, epochs=50)
model1.evaluate(x_test,y_test)

Epoch 1/50
52/52 [==============================] - 1s 17ms/step - loss: 2.3056 - accuracy: 0.0928 - f_score: 2.9695 - precision_m: 1.7710 - recall_m: 9.4638
Epoch 2/50
52/52 [==============================] - 1s 18ms/step - loss: 2.3017 - accuracy: 0.1146 - f_score: 2.6971 - precision_m: 1.5740 - recall_m: 9.5248
Epoch 3/50
52/52 [==============================] - 1s 17ms/step - loss: 2.2996 - accuracy: 0.1043 - f_score: 3.1240 - precision_m: 1.8761 - recall_m: 9.4184
Epoch 4/50
52/52 [==============================] - 1s 17ms/step - loss: 2.2896 - accuracy: 0.1631 - f_score: 2.5069 - precision_m: 1.4449 - recall_m: 9.5892
Epoch 5/50
52/52 [==============================] - 1s 17ms/step - loss: 2.2578 - accuracy: 0.1771 - f_score: 2.8271 - precision_m: 1.6634 - recall_m: 9.4853
Epoch 6/50
52/52 [==============================] - 1s 17ms/step - loss: 2.1843 - accuracy: 0.1953 - f_score: 3.2098 - precision_m: 1.9387 - recall_m: 9.4145
Epoch 7/50
52/52 [==============================] - 

[1.0171550512313843,
 0.6803874373435974,
 2.3442542552948,
 1.487420916557312,
 5.579272747039795]

In [112]:
from sklearn.metrics import classification_report
outx = model1.predict(x_test)
loss,acc,score,precision,recall= model1.evaluate(x_test,y_test)

13/13 [==============================] - 0s 8ms/step - loss: 1.0172 - accuracy: 0.6804 - f_score: 2.3443 - precision_m: 1.4874 - recall_m: 5.5793


In [114]:
def NN2():
    model = keras.Sequential([
    keras.layers.Dense(512, input_shape=(64*64,), activation='relu'),
    keras.layers.Dense(256, activation='relu'),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(10, activation='sigmoid')
    
])

    model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy',f_score,precision_m, recall_m])

    return model

In [115]:
model2 = NN2()
model2.fit(x_train, y_train, epochs=50)
model2.evaluate(x_test,y_test)

Epoch 1/50
52/52 [==============================] - 2s 26ms/step - loss: 2.3062 - accuracy: 0.0988 - f_score: 2.9776 - precision_m: 1.7744 - recall_m: 9.4695
Epoch 2/50
52/52 [==============================] - 1s 25ms/step - loss: 2.3025 - accuracy: 0.1013 - f_score: 2.8201 - precision_m: 1.6670 - recall_m: 9.5002
Epoch 3/50
52/52 [==============================] - 1s 19ms/step - loss: 2.3004 - accuracy: 0.1086 - f_score: 3.1344 - precision_m: 1.8958 - recall_m: 9.4549
Epoch 4/50
52/52 [==============================] - 1s 18ms/step - loss: 2.2892 - accuracy: 0.1201 - f_score: 3.1117 - precision_m: 1.8684 - recall_m: 9.4340
Epoch 5/50
52/52 [==============================] - 1s 18ms/step - loss: 2.1951 - accuracy: 0.1680 - f_score: 3.7224 - precision_m: 2.3396 - recall_m: 9.3645
Epoch 6/50
52/52 [==============================] - 1s 18ms/step - loss: 2.0902 - accuracy: 0.1868 - f_score: 3.1349 - precision_m: 1.8970 - recall_m: 9.2043
Epoch 7/50
52/52 [==============================] - 

[1.001624584197998,
 0.6997578740119934,
 2.1752123832702637,
 1.3967028856277466,
 4.973269939422607]

In [116]:
outx2 = model2.predict(x_test)
loss,acc,score,precision,recall= model2.evaluate(x_test,y_test)

13/13 [==============================] - 0s 7ms/step - loss: 1.0016 - accuracy: 0.6998 - f_score: 2.1752 - precision_m: 1.3967 - recall_m: 4.9733


In [106]:
def cross_valid1(x_values, y_values, k=5 ,Model = NN1()):
      scores = list()
      kfold = KFold(k, shuffle=True, random_state=1)
      for train_ix, test_ix in kfold.split(x_values):
        trainX, trainY, testX, testY = x_values[train_ix], y_values[train_ix], x_values[test_ix], y_values[test_ix]
        trainX = trainX.reshape(len(trainX),64*64)
        testX = testX.reshape(len(testX),64*64)
        crossValidation = Model
        crossValidation.fit(trainX, trainY, epochs=50)
        acc = crossValidation.evaluate(testX,testY)
        scores.append(acc)
      return scores

In [17]:
scoresNN1 , valueNN1 = cross_valid1(x_train,y_train,3,model1)


Epoch 1/50
35/35 [==============================] - 1s 16ms/step - loss: 0.9451 - accuracy: 0.6752 - f_score: 2.3726
Epoch 2/50
35/35 [==============================] - 1s 17ms/step - loss: 0.9266 - accuracy: 0.6879 - f_score: 2.3897
Epoch 3/50
35/35 [==============================] - 1s 17ms/step - loss: 0.9349 - accuracy: 0.6779 - f_score: 2.3780
Epoch 4/50
35/35 [==============================] - 1s 17ms/step - loss: 0.9233 - accuracy: 0.6888 - f_score: 2.3877
Epoch 5/50
35/35 [==============================] - 1s 16ms/step - loss: 0.9315 - accuracy: 0.6706 - f_score: 2.3500
Epoch 6/50
35/35 [==============================] - 1s 16ms/step - loss: 0.9187 - accuracy: 0.6970 - f_score: 2.3530
Epoch 7/50
35/35 [==============================] - 1s 16ms/step - loss: 0.9116 - accuracy: 0.7061 - f_score: 2.3346
Epoch 8/50
35/35 [==============================] - 1s 16ms/step - loss: 0.9003 - accuracy: 0.7106 - f_score: 2.3630
Epoch 9/50
35/35 [==============================] - 1s 16ms/step

In [19]:
print(scoresNN1)


[[0.8441534042358398, 0.7581818103790283, 2.134960412979126], [0.6144623756408691, 0.8054545521736145, 2.1238958835601807], [0.506182074546814, 0.8397085666656494, 1.9841796159744263]]


In [107]:
scoresNN2 = cross_valid1(x_train,y_train,3,model2)


Epoch 1/50
35/35 [==============================] - 1s 18ms/step - loss: 0.9990 - accuracy: 0.6278 - f_score: 2.1193
Epoch 2/50
35/35 [==============================] - 1s 19ms/step - loss: 0.8956 - accuracy: 0.6688 - f_score: 2.1391
Epoch 3/50
35/35 [==============================] - 1s 19ms/step - loss: 0.8939 - accuracy: 0.6588 - f_score: 2.1352
Epoch 4/50
35/35 [==============================] - 1s 19ms/step - loss: 0.9182 - accuracy: 0.6460 - f_score: 2.1085
Epoch 5/50
35/35 [==============================] - 1s 19ms/step - loss: 0.9119 - accuracy: 0.6515 - f_score: 2.1122
Epoch 6/50
35/35 [==============================] - 1s 19ms/step - loss: 0.8531 - accuracy: 0.6879 - f_score: 2.1415
Epoch 7/50
35/35 [==============================] - 1s 19ms/step - loss: 0.8360 - accuracy: 0.6879 - f_score: 2.1044
Epoch 8/50
35/35 [==============================] - 1s 19ms/step - loss: 0.8401 - accuracy: 0.6870 - f_score: 2.1240
Epoch 9/50
35/35 [==============================] - 1s 19ms/step

In [108]:
print(scoresNN2)


[[0.776707649230957, 0.7690908908843994, 1.9870457649230957], [0.5093985199928284, 0.8290908932685852, 1.9577386379241943], [0.32836490869522095, 0.8888888955116272, 1.915360927581787]]


In [22]:
from sklearn.svm import SVC
svm=SVC()
svm.fit(x_train,y_train)
svm.score(x_test,y_test)


0.8329297820823245

# **RGB WITH CNN**

In [ ]:
################################################RGB################################

In [23]:
dataset_path = "/content/drive/MyDrive/Dataset"
def get_img(dataset_path):
    img = imread(dataset_path)
    img = resize(img, (64, 64,3))
    return img

In [47]:
labels = listdir(dataset_path) # Geting labels
X = []
Y = []
for i, label in enumerate(labels):
    datas_path = dataset_path+'/'+label
    for data in listdir(datas_path):
        img = get_img(datas_path + '/' + data)
        X.append(img)
        Y.append(i)
X = np.array(X).astype('float32')
Y = np.array(Y).astype('float32')
X*=255.0
mean_x=X.mean()
x_train2, x_test2, y_train2, y_test2 = train_test_split(X, Y, test_size=0.2, random_state=42)

x_train2 = x_train2 -mean_x
x_train2 /=255.0
x_test2 = x_test2 -mean_x
x_test2 /=255.0

print(x_train2.shape)

(1649, 64, 64, 3)


In [117]:
def CNN():

      model = keras.Sequential([
          
           keras.layers.Conv2D(filters=32, kernel_size=(4, 4), activation='relu', input_shape=(64, 64 ,3)),
           keras.layers.MaxPooling2D((2, 2)),
          
           keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
           keras.layers.MaxPooling2D((2, 2)),
          
           keras.layers.Flatten(),
           keras.layers.Dense(64, activation='relu'),
           keras.layers.Dense(10, activation='sigmoid')
      ])

      model.compile(optimizer='adam',
                    loss='sparse_categorical_crossentropy',
                    metrics=['accuracy',f_score,precision_m, recall_m])
      return model


In [119]:
cnn=CNN()
cnn.fit(x_train2,y_train2,epochs=5)
cnn.evaluate(x_test2,y_test2)

Epoch 1/5
52/52 [==============================] - 10s 174ms/step - loss: 1.4082 - accuracy: 0.5167 - f_score: 3.0043 - precision_m: 1.8571 - recall_m: 7.9808
Epoch 2/5
52/52 [==============================] - 9s 176ms/step - loss: 0.5557 - accuracy: 0.8217 - f_score: 2.4746 - precision_m: 1.5947 - recall_m: 5.5668
Epoch 3/5
52/52 [==============================] - 9s 176ms/step - loss: 0.3814 - accuracy: 0.8787 - f_score: 2.3534 - precision_m: 1.4851 - recall_m: 5.7175
Epoch 4/5
52/52 [==============================] - 9s 177ms/step - loss: 0.2058 - accuracy: 0.9424 - f_score: 2.1423 - precision_m: 1.3596 - recall_m: 5.0859
Epoch 5/5
13/13 [==============================] - 1s 50ms/step - loss: 0.4333 - accuracy: 0.8789 - f_score: 1.9274 - precision_m: 1.2206 - recall_m: 4.5994


[0.4332619309425354,
 0.8789346218109131,
 1.9274147748947144,
 1.2206456661224365,
 4.599366664886475]

In [121]:
def cross_valid2(x_values, y_values, k=5 ,Model = CNN()):
      scores = list()
      kfold = KFold(k, shuffle=True, random_state=1)
      for train_ix, test_ix in kfold.split(x_values):
        trainX, trainY, testX, testY = x_values[train_ix], y_values[train_ix], x_values[test_ix], y_values[test_ix]
        crossValidation = Model
        crossValidation.fit(trainX, trainY, epochs=10)
        acc = crossValidation.evaluate(testX,testY)
        scores.append(acc)
      return scores,

In [122]:
scoresCNN = cross_valid2(x_train2,y_train2,3,cnn)


Epoch 1/10
35/35 [==============================] - 6s 177ms/step - loss: 0.0747 - accuracy: 0.9791 - f_score: 2.0273 - precision_m: 1.2907 - recall_m: 4.7609
Epoch 2/10
35/35 [==============================] - 6s 177ms/step - loss: 0.0339 - accuracy: 0.9854 - f_score: 1.9363 - precision_m: 1.2438 - recall_m: 4.3989
Epoch 3/10
35/35 [==============================] - 6s 175ms/step - loss: 0.0134 - accuracy: 0.9891 - f_score: 1.9018 - precision_m: 1.2147 - recall_m: 4.4148
Epoch 4/10
35/35 [==============================] - 6s 177ms/step - loss: 0.0149 - accuracy: 0.9836 - f_score: 1.8571 - precision_m: 1.1778 - recall_m: 4.4117
Epoch 5/10
35/35 [==============================] - 6s 177ms/step - loss: 0.0048 - accuracy: 0.9827 - f_score: 1.8260 - precision_m: 1.1621 - recall_m: 4.2823
Epoch 6/10
35/35 [==============================] - 6s 176ms/step - loss: 0.0020 - accuracy: 0.9809 - f_score: 1.8006 - precision_m: 1.1505 - recall_m: 4.1650
Epoch 7/10
35/35 [============================

In [43]:
from sklearn.svm import SVC
x_train2  = x_train2.reshape(len(x_train2),64*64*3)
x_test2 = x_test2.reshape(len(x_test2),64*64*3)
svm=SVC()
svm.fit(x_train2,y_train2)
svm.score(x_test2,y_test2)

0.837772397094431